In [1]:
from noise_control import *
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
rc_data = np.load("boptest_data.npz")
A = rc_data["A"]
B = rc_data["B"]
D = rc_data["D"]
R = rc_data["R"]
COP = rc_data["COP"]

In [3]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams["ps.fonttype"] = 42

In [4]:
sim_day = 7
Tsim = int(sim_day*24*4)
# noise_basic = np.array([45, 47, 52, 55, 57, 59, 60, 62, 66, 63, 
#                        62, 64, 63, 64, 63, 62, 61, 61, 67, 60, 58, 59, 56, 52, 49])
noise_basic = np.array([43, 42, 41.5, 42, 43, 42, 45, 52, 57,  
                       58, 64, 63, 64, 63, 62, 61, 61, 65, 60, 55, 48, 47, 44, 42])
nos_24_normal = np.repeat(noise_basic, 4)
nos_24_noisy = nos_24_normal + (nos_24_normal >= 60)*nos_24_normal*0.1 

nos_24_quite = nos_24_normal - (nos_24_normal >= 60)*nos_24_normal*0.05

noise_base = {"quite": nos_24_quite, "normal": nos_24_normal, "noisy": nos_24_noisy}

noise_pattern = "normal"

amb_nos_list = np.tile(noise_base[noise_pattern], sim_day+1)

In [5]:
eta_list_inv = np.logspace(-4, 1, 12)

energy_cost_list = []
noise_cost_list = []
noise_list = []
temp_list = []
obj_opt = ""
ele_price_pat = ""
build_type = ""
N = 4*8 ### prediction horizon is 12 hours
noise_cost_list = []
f_nos_list = []

In [6]:
#eta_list_res = np.logspace(-4, 0, 12)
eta_list_inv = np.logspace(-4, 1, 12)

energy_cost_list = []
noise_cost_list = []
noise_list = []
temp_list = []
obj_opt = ""
ele_price_pat = ""
build_type = ""
N = 4*8 ### prediction horizon is 12 hours
noise_cost_list = []
f_nos_list = []


###### closed-loop simulation #######
for eta in tqdm(eta_list_inv):
    hp_mpc = NoiseCtr(A,B,D,R, N = N, platform = "web", obj_opt = "inverse_penalty", ele_price_pat = "highly_dynamic", build_type = "normal")
    obj_opt = hp_mpc.obj_opt
    ele_price_pat = hp_mpc.ele_price_pat
    build_type = hp_mpc.build_type
    hp_mpc.init_state()
    hp_mpc.initialize_buff()
    hp_mpc.delta = 0
    for t in range(Tsim):
        hp_mpc.open_loop = False
        amb_nos = amb_nos_list[t:t+N]
      
        hp_mpc.get_forecast()
        hp_mpc.mpc_ctr(amb_nos, eta)
        hp_mpc.step()
        
    energy_cost_list.append(hp_mpc.energy_cost)
    
    noise_cost_list.append(hp_mpc.noise_cost)
    noise_list.append(hp_mpc.mixed_noise_list)
    temp_list.append(hp_mpc.ti_list) 
    f_nos_list.append(hp_mpc.f_nos_list)
#np.savez(f"{obj_opt}_{ele_price_pat}_{build_type}_{sim_day}.npz", noise_cost = noise_cost_list, energy_cost = energy_cost_list)

  0%|                                                                                           | 0/12 [22:23<?, ?it/s]

Unexpected error: {"message": "Unknown arguments: oveHeaPumY_u, oveHeaPumY_activate"}

Exiting!


SystemExit: 

In [ ]:
np.savez(f"{obj_opt}_{ele_price_pat}_{build_type}_{sim_day}.npz", 
         noise_cost = noise_cost_list, energy_cost = energy_cost_list, ti_list_res = temp_list, 
f_nos_list = f_nos_list, noise_list = noise_list)

## Plot simulation results

In [ ]:
#### compute the Lden value for the simulated data ######
day_penalty = 0
evening_penalty = 5
night_penalty = 10
penalty = np.concatenate((10*np.ones(7), np.zeros(12), 5*np.ones(4), 10*np.ones(1)))
penalty_list = penalty

quite_filter = np.ones(24)
quite_filter[8:22] = 0
L_quite_list = []
domination_list = []

Lden_list = []


cs_font = {"fontname":"Times New Roman"}
font_size = {"fontsize": 13}

########################## compute ambient Lden ###################
Lden_list_amb = []
for iter in range(len(eta_list_inv)):
    L_den = 0
    for day in range(sim_day): 
        L_orig = amb_nos_list[day*24*4:(day+1)*24*4]
        assert len(L_orig) == 24*4

        
        L_per_day = np.array([ 10*np.log10( 1/4*sum( 10**( np.array(L_orig[t:t+4])/10 ) ) ) for t in range(0, 24*4,4) ])

        L_per_day += penalty_list
        assert len(L_per_day) == 24
        
        L_nit = 10*np.log10( 1/8*(sum(10**(np.array(L_per_day[t]/10)) for t in range(7) ) + 10**(L_per_day[-1]/10) ) )
        L_eve = 10*np.log10( 1/4*(sum(10**(np.array(L_per_day[t]/10)) for t in range(19,23)) ) )
        L_day = 10*np.log10( 1/12*(sum(10**(np.array(L_per_day[t]/10)) for t in range(7,19)) ) )

        L_den += 10*np.log10(1/24*(12*(10**(L_day/10)) + 4*(10**(L_eve/10)) + 8*(10**(L_nit/10)) ))

    
    Lden_list_amb.append(L_den/sim_day)

#######################################################################

for iter in range(len(eta_list_inv)):
    L_den = 0
    L_quite = 0
    domi_time = 0
    for day in range(sim_day): 
        L_orig = noise_list[iter][day*24*4:(day+1)*24*4]
        assert len(L_orig) == 24*4

        L_per_day = np.array([ 10*np.log10( 1/4*sum(10**( np.array(L_orig[t:t+4])/10 )) ) for t in range(0, 24*4,4) ])

        L_quite_day = L_per_day*quite_filter
        L_quite_day = np.array(L_quite_day[L_quite_day >0])

        L_per_day += penalty_list
      
        L_nit = 10*np.log10( 1/8*(sum(10**(np.array(L_per_day[t]/10)) for t in range(7) ) + 10**(L_per_day[-1]/10) ) )
        L_eve = 10*np.log10( 1/4*(sum(10**(np.array(L_per_day[t]/10)) for t in range(19,23)) ) )
        L_day = 10*np.log10( 1/12*(sum(10**(np.array(L_per_day[t]/10)) for t in range(7,19)) ) )

        L_den += 10*np.log10(1/24*(12*(10**(L_day/10)) + 4*(10**(L_eve/10)) + 8*(10**(L_nit/10)) ))

        L_quite += 10*np.log10( 1/(len(L_quite_day)) *sum(10**(L_quite_day/10) ) )
 
        domi_time += sum( (f_nos_list[iter][day*24*4:(day+1)*24*4] - nos_24_normal ) > 0)/4
    
    Lden_list.append(L_den/sim_day)
    L_quite_list.append(L_quite/sim_day)
    domination_list.append(domi_time/sim_day)

###########  Plot Pareto Curves ###########
# plt.figure(figsize = (4,2.5))
# plt.plot(eta_list_inv, Lden_list)
# plt.scatter(eta_list_inv, Lden_list)
# plt.xscale("log")
# plt.xlabel("$\eta$", **cs_font, **font_size)
# plt.ylabel("$L_{den}$ (dB)", **cs_font, **font_size)
# plt.savefig(f"{hp_mpc.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc.obj_opt}_lden.pdf", bbox_inches = "tight")
#plt.title(f"{hp_mpc.ele_price_pat}_{hp_mpc.build_type}")


print("reduction of Lden is: ", Lden_list[0] - Lden_list[np.argmin(noise_cost_list)])
print("Lden reduction percentage is: ", (Lden_list[0] - Lden_list[np.argmin(noise_cost_list)])/(Lden_list[0] - Lden_list_amb[0]) )



# plt.figure(figsize = (4,2.5))
# plt.plot(eta_list_inv, L_quite_list)
# plt.scatter(eta_list_inv, L_quite_list)
# plt.ylabel("$L_{quite}$ (dB)")

print("L quite reduction is: ", L_quite_list[0] - L_quite_list[np.argmin(noise_cost_list)])

# plt.figure(figsize = (4,2.5))
# plt.plot(eta_list_inv, domination_list)
# plt.scatter(eta_list_inv, domination_list)
# plt.ylabel("domination time (h)")

print("domination time reduction is: ", domination_list[0] - domination_list[np.argmin(noise_cost_list)])

In [ ]:
#######################################################################

baseline1 = np.load("baseline1.npz")
noise_cost_baseline = baseline1["noise_cost1"]
energy_cost_baseline = baseline1["energy_cost"]
mixed_noise_baseline = baseline1["mixed_noise"]
true_noise_cost_baseline = baseline1["true_noise_cost"]


Lden_baseline = []
for _ in range(len(eta_list_inv)):
    L_den = 0
    for day in range(sim_day): 
        L_orig = mixed_noise_baseline[day*24*4:(day+1)*24*4]
        assert len(L_orig) == 24*4

        
        L_per_day = np.array([ 10*np.log10( 1/4*sum( 10**( np.array(L_orig[t:t+4])/10 ) ) ) for t in range(0, 24*4,4) ])

        L_per_day += penalty_list
        assert len(L_per_day) == 24
        
        L_nit = 10*np.log10( 1/8*(sum(10**(np.array(L_per_day[t]/10)) for t in range(7) ) + 10**(L_per_day[-1]/10) ) )
        L_eve = 10*np.log10( 1/4*(sum(10**(np.array(L_per_day[t]/10)) for t in range(19,23)) ) )
        L_day = 10*np.log10( 1/12*(sum(10**(np.array(L_per_day[t]/10)) for t in range(7,19)) ) )

        L_den += 10*np.log10(1/24*(12*(10**(L_day/10)) + 4*(10**(L_eve/10)) + 8*(10**(L_nit/10)) ))

    
    Lden_baseline.append(L_den/sim_day)

In [ ]:
print("noise cost reduction is: ", (noise_cost_list[0] - np.min(noise_cost_list))/noise_cost_list[0] )
print("energy cost increase is: ", (energy_cost_list[np.argmin(noise_cost_list)] - energy_cost_list[0])
      /energy_cost_list[0] )

In [ ]:
true_noise_cost = []
for iter in range(len(eta_list_inv)):
    noise_residule = 0
    for t in range(Tsim):
        noise_residule += (noise_list[iter][t] - amb_nos_list[t])
    true_noise_cost.append(noise_residule)

print("real noise cost reduction is: ", (true_noise_cost[0] - true_noise_cost[np.argmin(noise_cost_list)])/true_noise_cost[0] )

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MaxNLocator

lw = {"linewidth":2}
plt.rcParams["mathtext.fontset"] = "stix"
cs_font = {"fontname":"Times New Roman"}
font_size = {"fontsize": 12}
font_size_leg = {"fontsize", 10}


fig, ax = plt.subplots(1,4,figsize = (13.5,2.6), constrained_layout = True)

# ax[0].plot(np.array(energy_cost_list), np.array(noise_cost_list))
# sc1 = ax[0].scatter(np.array(energy_cost_list), np.array(noise_cost_list))
# ax[0].set_xlabel("energy cost (Euro)", **cs_font, **font_size)
# ax[0].set_ylabel("noise cost in Eq.(7)", **cs_font, **font_size)
# ax[0].set_title("(a)", loc = "left")
# #ax[0].set_ylim([200,280])

# ## baseline plot

# ax_inset = ax[0].inset_axes([0.2,0.7,0.2,0.2])
# sc2 = ax_inset.scatter(energy_cost_baseline, noise_cost_baseline, marker = "^", color ="r")
# ax_inset.set_ylim(400, 425)  
# ax_inset.set_xlim(34, 35)

# ax[0].legend([sc1, sc2], ["Pareto curve", "baseline"], frameon = False)

# # 添加断裂标记线
# d = .015  # 断裂标记的长度
# kwargs = dict(transform=ax[0].transAxes, color='k', clip_on=False)
# ax[0].plot((-d, +d), (-d, +d), **kwargs)
# ax[0].plot((1 - d, 1 + d), (-d, +d), **kwargs)

sca1 = ax[0].scatter(energy_cost_list, true_noise_cost)
ax[0].plot(energy_cost_list, true_noise_cost)
ax[0].set_ylabel("real noise cost (dB)", **cs_font, **font_size)
ax[0].set_xlabel("energy cost (Euro)", **cs_font, **font_size)
ax[0].set_xlim([33.9,37.6])

ax_twin = ax[0].twinx()
ax_twin.plot(np.array(energy_cost_list), np.array(noise_cost_list), "--", c = "orange",**lw)
sca2 = ax_twin.scatter(np.array(energy_cost_list), np.array(noise_cost_list), marker = "*", c = "orange")
ax_twin.set_ylim([195,285])
#ax_twin.legend([sca1, sca2], ["real noise cost", "$J_n$ in Eq.(9)"], frameon = False)

ax_twin.set_xlabel("energy cost (Euro)", **cs_font, **font_size)
ax_twin.set_ylabel("$J_n$ in Eq.(7)", **cs_font, **font_size)
ax_twin.set_title("(a)", loc = "left")


# 添加断裂标记线
# d = .015  # 断裂标记的长度
# kwargs = dict(transform=ax[0].transAxes, color='k', clip_on=False)
# ax[0].plot((-d, +d), (-d, +d), **kwargs)
# ax[0].plot((1 - d, 1 + d), (-d, +d), **kwargs)
#################
base1 = ax[0].scatter(energy_cost_baseline, true_noise_cost_baseline, marker="o", edgecolors='red', facecolors='none', s=40)
#base2 = ax_twin.scatter(energy_cost_baseline, noise_cost_baseline, marker='^', edgecolors='red', facecolors='none', s=50)
ax_twin.legend([sca1, sca2, base1], ["real noise cost", "$J_n$ in Eq.(7)","baseline noise cost"],
              frameon = False)



##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ax[1].plot(eta_list_inv, Lden_list)
scatter1 = ax[1].scatter(eta_list_inv, Lden_list)
ax[1].set_xscale("log")
ax[1].set_xlabel("$\eta$", **cs_font, **font_size)
ax[1].set_ylabel("$L_{\mathrm{den}}$ (dB)", **cs_font, **font_size)

ax[1].set_title("(b)", loc = "left")

### baseline plot
#ax[1].scatter(eta_list_inv, Lden_baseline, color = "r")

ax_new = ax[1].twinx()
# ax_new.set_ylim([200,1850])
ax_new.plot(eta_list_inv, noise_cost_list, "orange")
scatter2 = ax_new.scatter(eta_list_inv, noise_cost_list, c = "orange", marker = "*")

ax_new.set_ylabel("noise cost in Eq.(7)", **font_size, **cs_font)
ax_new.legend([scatter1, scatter2], ["$L_{\mathrm{den}}$", "noise cost"], frameon = False, loc = 3)
ax_new.set_ylim([195,285])

##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ax[2].plot(eta_list_inv, L_quite_list)
sc1 = ax[2].scatter(eta_list_inv, L_quite_list)
ax[2].set_ylabel("$L_{\mathrm{quiet}}$ (dB)", **cs_font, **font_size)
ax[2].set_xscale("log")
ax[2].set_xlabel("$\eta$", **cs_font, **font_size)

ax_2 = ax[2].twinx()
ax_2.plot(eta_list_inv, domination_list, color = "orange")
sc2 = ax_2.scatter(eta_list_inv, domination_list, color = "orange", marker = "*")
ax_2.set_ylabel("domination time (h)", **cs_font, **font_size)

ax_2.legend([sc1,sc2], ["$L_{\mathrm{quiet}}$", "domination time"], frameon = False, loc = 3)
ax_2.set_title("(c)", loc = "left")

ax[3].scatter(eta_list_inv, energy_cost_list, )
ax[3].plot(eta_list_inv, energy_cost_list)
ax[3].set_xscale("log")
ax[3].set_xlabel("$\eta$", **cs_font, **font_size)
ax[3].set_ylabel("energy cost (Euro)", **cs_font, **font_size)
ax[3].set_title("(d)", loc = "left")
ax[3].legend(["energy cost"], frameon = False)
#ax[3].set_ylim([34,35.6])
plt.savefig(f"{hp_mpc.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc.obj_opt}_all_base_final_new.pdf", bbox_inches = "tight")
#fig.tight_layout()

In [ ]:
from brokenaxes import brokenaxes
from matplotlib.gridspec import GridSpec
import numpy as np
fig = plt.figure(figsize = (12.1,2.7))
sps1, sps2, sps3 = GridSpec(1,3,width_ratios = [1,1,1])

bax = brokenaxes(ylims=((200, 280), (400,425)), subplot_spec=sps1)
bax.plot(np.array(energy_cost_list), np.array(noise_cost_list))
bax.scatter(np.array(energy_cost_list), np.array(noise_cost_list))

bax.scatter(energy_cost_baseline, noise_cost_baseline, marker = "^", color ="r")

In [ ]:
np.savez(f"{obj_opt}_{ele_price_pat}_{build_type}_{sim_day}.npz", 
         noise_cost = noise_cost_list, energy_cost = energy_cost_list, 
ti_list = temp_list, f_nos_list = f_nos_list, noise_list = noise_list)

# cs_font = {"fontname":"Times New Roman"}
# font_size = {"fontsize": 12}
# plt.figure(figsize = (4,2.5))
# plt.plot(np.array(energy_cost_list), np.array(noise_cost_list))
# plt.scatter(np.array(energy_cost_list), np.array(noise_cost_list))
# plt.xlabel("energy cost (Euro)", **cs_font, **font_size)
# plt.ylabel("noise cost", **cs_font, **font_size)
# plt.savefig(f"{hp_mpc.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc.obj_opt}.pdf", bbox_inches = "tight")

cs_font = {"fontname":"Times New Roman"}
font_size = {"fontsize": 13}

fig, ax = plt.subplots(2,1,figsize = (4.5,6.5))
#plt.subplot(211)
ax[0].plot(np.array(energy_cost_list), np.array(noise_cost_list))
ax[0].scatter(np.array(energy_cost_list), np.array(noise_cost_list))
ax[0].set_xlabel("energy cost (Euro)", **cs_font, **font_size)
ax[0].set_ylabel("noise cost (dB)", **cs_font, **font_size)
ax[0].set_title("(a)", loc = "left")

ax[1].plot(eta_list_inv, Lden_list)
st1 = ax[1].scatter(eta_list_inv, Lden_list)
ax[1].set_xscale("log")
ax[1].set_xlabel("$\eta$", **cs_font, **font_size)
ax[1].set_ylabel("$L_{den}$ (dB)", **cs_font, **font_size)
ax[1].set_title("(b)", loc = "left")
ax_new = ax[1].twinx()
ax_new.plot(eta_list_inv, noise_cost_list, "orange")
st2 = ax_new.scatter(eta_list_inv, noise_cost_list, c = "orange", marker = "*")
ax_new.legend([st1, st2], ["$L_{den}$", "noise cost"], frameon = False )
ax_new.set_ylabel("noise cost", **font_size, **cs_font)
plt.savefig(f"{hp_mpc.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc.obj_opt}_both_new.pdf", bbox_inches = "tight")

print("noise cost reduction is: ", (np.max(noise_cost_list) - np.min(noise_cost_list))/np.max(noise_cost_list) )
print("energy cost increase is: ", (energy_cost_list[np.argmin(noise_cost_list)] - energy_cost_list[np.argmax(noise_cost_list)])
      /energy_cost_list[np.argmax(noise_cost_list)] )

### Computation of average noise during quiet time

In [ ]:
# len(noise_list[1])/24/4
plt.plot(noise_list[0][:24*4])
plt.plot(noise_list[-1][:24*4],"r")
# plt.plot(nos_24_quite, "g")

In [ ]:
#### compute the Lden value for the simulated data ######
day_penalty = 0
evening_penalty = 5
night_penalty = 10
penalty = np.concatenate((10*np.ones(7), np.zeros(12), 5*np.ones(4), 10*np.ones(1)))
penalty_list = penalty

quite_filter = np.ones(24)
quite_filter[8:20] = 0
L_quite_list = []

Lden_list = []

for iter in range(len(eta_list_inv)):
    L_den = 0
    L_quite = 0
    for day in range(sim_day): 
        L_orig = noise_list[iter][day*24*4:(day+1)*24*4]

        
        L_per_day = np.array([ 10*np.log10( 1/4*(sum(10**(L_orig[t:t+4]))) ) for t in range(0, 24*4,4) ])
        #L_per_day = np.array([sum(L_orig[t:t+4])/4 for t in range(0,24*4,4)])

        L_quite_day = L_per_day*quite_filter
        
        L_per_day += penalty_list
        
        L_nit = (np.sum(L_per_day[t] for t in range(7)) + L_per_day[-1])/8
        L_eve = (np.sum(L_per_day[t] for t in range(19,23)))/4
        L_day = (np.sum(L_per_day[t] for t in range(7,19)) )/12
        # print(L_nit,L_eve,L_day)
        L_den += 10*np.log10(1/24*(12*(10**(L_day/10)) + 4*(10**(L_eve/10)) + 8*(10**(L_nit/10)) ))
        # print(L_den)
        L_quite += np.sum(L_quite_day)/np.sum(quite_filter)
    
    Lden_list.append(L_den/sim_day)
    L_quite_list.append(L_quite/sim_day)


plt.figure(figsize = (4,2.5))
plt.plot(eta_list_res, Lden_list)
plt.scatter(eta_list_res, Lden_list)
plt.xscale("log")
plt.xlabel("$\eta$", **cs_font, **font_size)
plt.ylabel("$L_{den}$ (dB)", **cs_font, **font_size)
#plt.savefig(f"{hp_mpc.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc.obj_opt}_lden.pdf", bbox_inches = "tight")
#plt.title(f"{hp_mpc.ele_price_pat}_{hp_mpc.build_type}")
#hp_mpc.P_max

print("reduction of Lden is: ", Lden_list[0] - np.min(Lden_list))
print("Lden reduction percentage is: ", (Lden_list[0] - np.min(Lden_list))/(Lden_list[0] - Lden_list_amb[0]) )

plt.figure()
plt.plot(L_quite_list)

## Simulation for Residule Noise Cost

In [ ]:
eta_list_res = np.logspace(-6, -1, 12)

energy_cost_list_res = []
noise_cost_list_res = []
noise_list_res = []
temp_list_res = []
obj_opt_res = "ambient_res"
ele_price_pat_res = "highly_dynamic"
N = 4*8 ### prediction horizon is 12 hours
noise_cost_list_res = []
build_type_res = "normal"
###### open-loop simulation #######
for eta in tqdm(eta_list_inv):
    hp_mpc_res = NoiseCtr(A,B,D,R, N = N, platform = "local", obj_opt = obj_opt_res, ele_price_pat = ele_price_pat_res,
                          build_type = build_type_res)
    obj_opt_res = hp_mpc_res.obj_opt
    ele_price_pat_res = hp_mpc_res.ele_price_pat
    build_type_res = hp_mpc_res.build_type
    hp_mpc_res.init_state()
    hp_mpc_res.initialize_buff()
    hp_mpc_res.delta = 0
    for t in range(Tsim):
        hp_mpc_res.open_loop = False
        amb_nos = amb_nos_list[t:t+N]
      
        hp_mpc_res.get_forecast()
        hp_mpc_res.mpc_ctr(amb_nos, eta)
        hp_mpc_res.step()
        
    energy_cost_list_res.append(hp_mpc_res.energy_cost)
    
    noise_cost_list_res.append(hp_mpc_res.noise_cost)
    noise_list_res.append(hp_mpc_res.mixed_noise_list)
    temp_list_res.append(hp_mpc_res.ti_list) 
    
#np.savez(f"{obj_opt}_{ele_price_pat}_{build_type}_{sim_day}.npz", noise_cost = noise_cost_list, energy_cost = energy_cost_list)

In [ ]:
np.savez(f"{obj_opt_res}_{ele_price_pat_res}_{build_type_res}_{sim_day}.npz", 
         noise_cost = noise_cost_list_res, energy_cost = energy_cost_list_res, ti_list_res = hp_mpc_res.ti_list, f_nos_list = hp_mpc_res.f_nos_list,noise_list = noise_list_res)

cs_font = {"fontname":"Times New Roman"}
font_size = {"fontsize": 13}

plt.figure(figsize = (4.5,6.5))
plt.subplot(211)
plt.plot(np.array(energy_cost_list_res), np.array(noise_cost_list_res))
plt.scatter(np.array(energy_cost_list_res), np.array(noise_cost_list_res))
plt.xlabel("energy cost (Euro)", **cs_font, **font_size)
plt.ylabel("noise cost (dB)", **cs_font, **font_size)
plt.title("(a)", loc = "left")

plt.subplot(212)
plt.plot(eta_list_res, Lden_list)
plt.scatter(eta_list_res, Lden_list)
plt.xscale("log")
plt.xlabel("$\eta$", **cs_font, **font_size)
plt.ylabel("$L_{den}$ (dB)", **cs_font, **font_size)
plt.title("(b)", loc = "left")
#plt.savefig(f"{hp_mpc_res.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc_res.obj_opt}.pdf", bbox_inches = "tight")
#hp_mpc.P_max
#plt.savefig(f"{hp_mpc_res.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc_res.obj_opt}_both.pdf", bbox_inches = "tight")

print("noise cost reduction is: ", (np.max(noise_cost_list_res) - np.min(noise_cost_list_res))/np.max(noise_cost_list_res) )
print("energy cost increase is: ", (energy_cost_list_res[np.argmin(noise_cost_list_res)] - energy_cost_list_res[np.argmax(noise_cost_list_res)])
      /energy_cost_list_res[np.argmax(noise_cost_list_res)] )

In [ ]:
plt.plot(noise_list_res[0][:24*4])
plt.plot(noise_list_res[-1][:24*4],"r")

In [ ]:

cs_font = {"fontname":"Times New Roman"}
font_size = {"fontsize": 13}

fig, ax = plt.subplots(2,1,figsize = (4.5,6.5))
#plt.subplot(211)
ax[0].plot(np.array(energy_cost_list_res), np.array(noise_cost_list_res))
ax[0].scatter(np.array(energy_cost_list_res), np.array(noise_cost_list_res))
ax[0].set_xlabel("energy cost (Euro)", **cs_font, **font_size)
ax[0].set_ylabel("noise cost (dB)", **cs_font, **font_size)
ax[0].set_title("(a)", loc = "left")

ax[1].plot(eta_list_res, Lden_list_res)
scatter1 = ax[1].scatter(eta_list_res, Lden_list_res)
ax[1].set_xscale("log")
ax[1].set_xlabel("$\eta$", **cs_font, **font_size)
ax[1].set_ylabel("$L_{den}$ (dB)", **cs_font, **font_size)
ax[1].set_title("(b)", loc = "left")

ax_new = ax[1].twinx()
ax_new.set_ylim([250,1650])
ax_new.plot(eta_list_res, noise_cost_list_res, "orange")
scatter2 = ax_new.scatter(eta_list_res, noise_cost_list_res, c = "orange", marker = "*")
ax_new.set_ylabel("noise cost (dB)", **font_size, **cs_font)
ax_new.legend([scatter1, scatter2], ["$L_{den}$", "noise cost"], frameon = False)

plt.savefig(f"{hp_mpc_res.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc_res.obj_opt}_both_new.pdf", bbox_inches = "tight")

In [ ]:
#### compute the Lden value for the simulated data ######
day_penalty = 0
evening_penalty = 5
night_penalty = 10
penalty = np.concatenate((10*np.ones(7), np.zeros(12), 5*np.ones(4), 10*np.ones(1)))
penalty_list = penalty

quite_filter = np.ones(24)
quite_filter[8:20] = 0
L_quite_list_res = []
domination_list = []

Lden_list_res = []
for iter in range(len(eta_list_res)):
    L_den = 0
    L_quite = 0

    for day in range(sim_day): 
        L_orig = noise_list_res[iter][day*24*4:(day+1)*24*4]
        assert len(L_orig) == 24*4
        #L_per_day = np.array([sum(L_orig[t:t+4])/4 for t in range(0,24*4,4)])
        
        L_per_day = np.array([ 10*np.log10( 1/4*sum( 10**( np.array(L_orig[t:t+4])/10 ) ) ) for t in range(0, 24*4,4) ])

        L_quite_day = L_per_day*quite_filter

        L_per_day += penalty_list
      
        # L_nit = (np.sum(L_per_day[t] for t in range(7)) + L_per_day[-1])/8
        # L_eve = (np.sum(L_per_day[t] for t in range(19,23)))/4
        # L_day = (np.sum(L_per_day[t] for t in range(7,19)) )/12
        L_nit = 10*np.log10( 1/8*(sum(10**(np.array(L_per_day[t]/10)) for t in range(7) ) + 10**(L_per_day[-1]/10) ) )
        L_eve = 10*np.log10( 1/4*(sum(10**(np.array(L_per_day[t]/10)) for t in range(19,23)) ) )
        L_day = 10*np.log10( 1/12*(sum(10**(np.array(L_per_day[t]/10)) for t in range(7,19)) ) )

        
        # print(L_nit,L_eve,L_day)
        L_den += 10*np.log10(1/24*(12*(10**(L_day/10)) + 4*(10**(L_eve/10)) + 8*(10**(L_nit/10)) ))
       
        L_quite += np.sum(L_quite_day)/np.sum(quite_filter)
    
    Lden_list_res.append(L_den/sim_day)
    L_quite_list_res.append(L_quite/sim_day)


plt.figure(figsize = (4,2.5))
plt.plot(eta_list_res, Lden_list_res)
plt.scatter(eta_list_res, Lden_list_res)
plt.xscale("log")
plt.xlabel("$\eta$", **cs_font, **font_size)
plt.ylabel("$L_{den}$ (dB)", **cs_font, **font_size)
#plt.savefig(f"{hp_mpc_res.open_loop}_{sim_day}_{noise_pattern}_{hp_mpc_res.obj_opt}_lden.pdf", bbox_inches = "tight")
#plt.title(f"{hp_mpc.ele_price_pat}_{hp_mpc.build_type}")
#hp_mpc.P_max

print("reduction of Lden is: ", Lden_list_res[0] - np.min(Lden_list_res))
print("Lden reduction percentage is: ", (Lden_list_res[0] - np.min(Lden_list_res))/(Lden_list_res[0] - Lden_list_amb[0]) )

plt.figure()
plt.plot(L_quite_list_res)

In [ ]:
Lden_list_amb = []
for iter in range(len(eta_list_res)):
    L_den = 0
    for day in range(sim_day): 
        L_orig = amb_nos_list[day*24*4:(day+1)*24*4]
        assert len(L_orig) == 24*4
        L_per_day = np.array([sum(L_orig[t:t+4])/4 for t in range(0,24*4,4)])

        assert len(L_per_day) == 24
        L_per_day += penalty_list
        
        L_nit = (np.sum(L_per_day[t] for t in range(7)) + L_per_day[-1])/8
        L_eve = (np.sum(L_per_day[t] for t in range(19,23)))/4
        L_day = (np.sum(L_per_day[t] for t in range(7,19)) )/12
        # print(L_nit,L_eve,L_day)
        L_den += 10*np.log10(1/24*(12*(10**(L_day/10)) + 4*(10**(L_eve/10)) + 8*(10**(L_nit/10)) ))
        # print(L_den)
    
    Lden_list_amb.append(L_den/sim_day)


plt.figure(figsize = (4,2.5))
plt.plot(eta_list_res, Lden_list_amb)
plt.scatter(eta_list_res, Lden_list_amb)
plt.xscale("log")
plt.xlabel("$\eta$", **cs_font, **font_size)
plt.ylabel("$L_{den}$ (dB)", **cs_font, **font_size)

## Plot ambient noise pattern

In [ ]:
cs_font = {"fontname":"Times New Roman"}
font_size = {"fontsize": 12}

plt.figure(figsize=(4,2.5))
x_label = np.linspace(0,24,len(noise_base[noise_pattern]) )
plt.plot(x_label, noise_base[noise_pattern])
plt.ylim([40,68])

plt.xlabel("Time (h)", **cs_font, **font_size)
plt.ylabel("Ambient Noise (dB)", **cs_font, **font_size)
plt.savefig("amb_noise.pdf", bbox_inches = "tight")

In [ ]:
plt.plot(energy_cost_list)
plt.xlabel("iterations of $\eta$")
plt.ylabel("energy cost")

In [ ]:
plt.plot(noise_cost_list)
plt.xlabel("iterations of $\eta$")
plt.ylabel("noise_residule")

In [ ]:
plt.plot(np.array(noise_list[0][:24*4]),"g")
plt.plot(amb_nos_list[:24*4],'r')
plt.plot(noise_list[-1][:24*4],"b")
plt.plot(hp_mpc.f_nos_list[:24*4])

In [ ]:
plt.plot(hp_mpc.f_nos_list)

In [ ]:
np.max(np.array(noise_list[-1]) - amb_nos_list[:24*4])

In [ ]:
plt.plot(temp_list[1])
plt.plot(temp_list[-1],"r")
plt.plot(19*np.ones(hp_mpc.N))

In [ ]:
plt.plot(hp_mpc.ele_price)

In [ ]:
a = [1,2]
b = 5
a.append(b)
a

In [ ]:
a = np.array([1,2])

for t in range(3):
    print(sum(a[t+k] for k in range(2-t)))

In [ ]:
def auto_tune(a = 1,[true, false,   ]b,c):
    for 
    func.fit(data)
    func.test(data)